In [9]:
import pandas as pd

# concat historical data and rename columns
pos2024 = pd.read_excel('Blackstone Overall Sales Metric 2024.xlsx',sheet_name='POS')
pos2025 = pd.read_excel('Blackstone Overall Sales Metric 2025.xlsx',sheet_name='POS')
pos = pd.concat([pos2024,pos2025])
pos.rename(columns={'Retailer':'Customer Name','Material ID':'Item','Netsuite Category':'Product Group',
					'Sales Units (SUM)':'POS Quantity'},inplace=True)

# Standardize Customer Name
pos['Customer Name'] = pos['Customer Name'].str.replace('Blackstone Sales','Blackstone Website')
pos['Customer Name'] = pos['Customer Name'].str.replace("Dick's Sporting Goods","Dicks")
pos['Customer Name'] = pos['Customer Name'].str.replace('TSC','Tractor Supply')

# creating Date column
pos['Date'] = pd.to_datetime(pos['Year'].astype(str) + 'W' + pos['Week'].astype(str) + '1', format='%YW%W%w')

# group and sort dataset
pos_group = pos.groupby(['Customer Name','Item','Product Group','Date'])['POS Quantity'].sum()
pos_group = pd.DataFrame(pos_group)

# establish LW POS Quantity
pos_group['LW POS Quantity'] = pos_group.groupby(level=0)['POS Quantity'].shift(1).fillna(0)

# save to csv for uploading to Domo via CSV Workbench
pos_group.to_csv('Weekly POS Sales Report from Darren.csv')

pos_group.head(20)

POS Quantity  LW POS Quantity
Customer Name Item Product Group     Date                                     
AAFES         1131 Covers/Bags - GRD 2024-02-19             1              0.0
                                     2024-06-10             1              1.0
                                     2025-04-14             1              1.0
                                     2025-04-28             1              1.0
                                     2025-05-12             1              1.0
              1182 Covers/Bags - GRD 2024-05-13             1              1.0
                                     2024-05-27             1              1.0
                                     2025-06-02             1              1.0
              1550 Griddles          2025-02-10             1              1.0
                                     2025-03-24             1              1.0
                                     2025-03-31             1              1.0
              1720 Covers/Bags - GRD 2024-01-01             1              1.0
                                     2024-01-29             1              1.0
                                     2024-04-29             1              1.0
                                     2024-05-27             1              1.0
                                     2024-06-17             1              1.0
                                     2024-06-24             1              1.0
                                     2024-07-08             1              1.0
                                     2024-07-29             1              1.0
                                     2024-12-02             1              1.0

In [26]:
pos2024.head()

,Year,Week,Retailer,Material ID,Netsuite Category,Sales Units (SUM)
0,2024,1,AAFES,1720,Covers/Bags - GRD,1
1,2024,1,AAFES,1724,Covers/Bags - GRD,2
2,2024,1,AAFES,1935,Griddles,1
3,2024,1,AAFES,1968,Griddles,1
4,2024,1,AAFES,4114,Consumables,6


In [ ]:
CASE 

WHEN `POS Period Comparison` = 'Last Week over Previous Week' THEN (CASE WHEN WEEK(`Date`) = WEEK(CURDATE()) - 1 AND YEAR(`Date`) = YEAR(CURDATE()) AND DAYOFYEAR(`Date`) <= DAYOFYEAR(CURDATE()) THEN `POS Quantity` END)

WHEN `POS Period Comparison` = 'MTD over LMTD' THEN (CASE WHEN MONTH(`Date`) = MONTH(CURDATE()) AND YEAR(`Date`) = YEAR(CURDATE()) AND DAYOFYEAR(`Date`) <= DAYOFYEAR(CURDATE()) THEN `POS Quantity` END)

WHEN `POS Period Comparison` = 'QTD over LQTD' THEN (CASE WHEN QUARTER(`Date`) = QUARTER(CURDATE()) AND YEAR(`Date`) = YEAR(CURDATE()) AND DAYOFYEAR(`Date`) <= DAYOFYEAR(CURDATE()) THEN `POS Quantity` END)

WHEN `POS Period Comparison` = 'YTD over LYTD' THEN (CASE WHEN YEAR(`Date`) = YEAR(CURDATE()) AND DAYOFYEAR(`Date`) <= DAYOFYEAR(CURDATE()) THEN `POS Quantity` END)

WHEN `POS Period Comparison` = 'WTD over same WTD LY' THEN (CASE WHEN WEEK(`Date`) = WEEK(CURDATE()) AND YEAR(`Date`) = YEAR(CURDATE()) AND DAYOFYEAR(`Date`) <= DAYOFYEAR(CURDATE()) THEN `POS Quantity` END)

WHEN `POS Period Comparison` = 'MTD over same MTD LY' THEN (CASE WHEN MONTH(`Date`) = MONTH(CURDATE()) AND YEAR(`Date`) = YEAR(CURDATE()) AND DAYOFYEAR(`Date`) <= DAYOFYEAR(CURDATE()) THEN `POS Quantity` END)

WHEN `POS Period Comparison` = 'QTD over same QTD LY' THEN (CASE WHEN QUARTER(`Date`) = QUARTER(CURDATE()) AND YEAR(`Date`) = YEAR(CURDATE()) AND DAYOFYEAR(`Date`) <= DAYOFYEAR(CURDATE()) THEN `POS Quantity` END)

END